# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names()) 

['admin', 'config', 'local', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review the collections in our new database
establishments = db["establishments"] 

In [8]:
# review a document in the establishments collection
pprint(establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('681d409da9f3fd8363f1c172'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [9]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": 1,
    "AddressLine1": "123 Greenwich High Road",
    "AddressLine2": "Greenwich",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE10 8JA",
    "Phone": "",
    "RatingValue": "AwaitingInspection",          # not rated yet
    "RatingKey": "fhrs_awaitinginspection_en-gb",
    "RatingDate": None,
    "LocalAuthorityName": "Greenwich",
    "scores": {                                   # placeholders
        "Hygiene": None,
        "Structural": None,
        "ConfidenceInManagement": None
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": 0.006305,
        "latitude": 51.477928
    },
    "RightToReply": "",
    "NewRatingPending": True
}


In [11]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)
print(f"Inserted document ID: {insert_result.inserted_id}")


Inserted document ID: 681d4992016462a9c7ba4f0d


In [12]:
# Check that the new restaurant was inserted
from pprint import pprint
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': '123 Greenwich High Road',
 'AddressLine2': 'Greenwich',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Greenwich',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE10 8JA',
 'RatingDate': None,
 'RatingKey': 'fhrs_awaitinginspection_en-gb',
 'RatingValue': 'AwaitingInspection',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('681d4992016462a9c7ba4f0d'),
 'geocode': {'latitude': 51.477928, 'longitude': 0.006305},
 'scores': {'ConfidenceInManagement': None,
            'Hygiene': None,
            'Structural': None}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_info = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"_id": 0, "BusinessType": 1, "BusinessTypeID": 1}
)
pprint(business_type_info)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID
correct_id = business_type_info["BusinessTypeID"]

update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": correct_id}}
)
print(f"Matched: {update_result.matched_count}, Modified: {update_result.modified_count}")


Matched: 1, Modified: 0


In [15]:
# Confirm that the new restaurant was updated
pprint(
    establishments.find_one(
        {"BusinessName": "Penang Flavours"},
        {"_id": 0, "BusinessName": 1, "BusinessType": 1, "BusinessTypeID": 1}
    )
)


{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [16]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Documents in Dover before deletion: {dover_count}")


Documents in Dover before deletion: 994


In [17]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Deleted documents: {delete_result.deleted_count}")


Deleted documents: 994


In [18]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Documents in Dover after deletion: {remaining_dover}")


Documents in Dover after deletion: 0


In [19]:
# Check that other documents remain with 'find_one'
from pprint import pprint
pprint(
    establishments.find_one(
        {"LocalAuthorityName": {"$ne": "Dover"}},
        {"_id": 0, "BusinessName": 1, "LocalAuthorityName": 1}
    )
)


{'BusinessName': 'Wear Bay Bowls Club',
 'LocalAuthorityName': 'Folkestone and Hythe'}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Change the data type from String to Decimal for longitude and latitude
coords_result = establishments.update_many(
    {},
    [
        {"$set": {
            "geocode.longitude": {"$toDouble": "$geocode.longitude"},
            "geocode.latitude":  {"$toDouble": "$geocode.latitude"}
        }}
    ]
)
print(f"Modified (lat/long): {coords_result.modified_count}")

Modified (lat/long): 38785


Use `update_many` to convert `RatingValue` to integer numbers.

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4092, 'nModified': 4092, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Change the data type from String to Integer for RatingValue
ratings_result = establishments.update_many(
    {"RatingValue": {"$type": "string"}},        # only the numeric strings remain
    [
        {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
    ]
)
print(f"Modified (ratings): {ratings_result.modified_count}")


Modified (ratings): 34694


In [23]:
# Check that the coordinates and rating value are now numbers
from pprint import pprint

pprint(
    establishments.find_one(
        {"RatingValue": {"$type": "int"}},       # grab a sample that is definitely numeric
        {"_id": 0, "BusinessName": 1,
         "RatingValue": 1,
         "geocode.longitude": 1,
         "geocode.latitude": 1}
    )
)


{'BusinessName': 'Wear Bay Bowls Club',
 'RatingValue': 4,
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408}}
